# Deep Learning for Healthcare Team 55 Project
### Zeeshan Haidry, Hamza Mahmood, Nithin Nathan

Team 55 GitHub Repo: https://github.com/zeeshanhaidry/cs598dlh-team55

Project based on:
Fayyaz H, Strang A, Beheshti R. Bringing At-home Pediatric Sleep Apnea Testing Closer to Reality: A Multi-modal Transformer Approach. Proc Mach Learn Res. 2023 Aug;219:167-185. PMID: 38344396; PMCID: PMC10854997.

Original GitHub: https://github.com/healthylaife/Pediatric-Apnea-Detection

In [ ]:
# instead of drive, we will be using uofi box

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# download dependencies

!pip install biosppy
!pip install boxsdk
!pip install mne==1.0
!pip install tensorflow
!pip install tensorflow-addons
!pip install gdown

  Using cached boxsdk-3.9.2-py2.py3-none-any.whl (139 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached mne-1.0.0-py3-none-any.whl (7.5 MB)


In [ ]:
# authenticate to connect to uofi box

# Data was uploaded to box from sleepdata website. (Link provided in data section)
# Commented to allow remaining of project to run since secrets might not be setup.

from google.colab import userdata
from boxsdk import Client, OAuth2, CCGAuth
from boxsdk.object import file, folder
from pprint import pformat
import json

# CLIENT_ID = userdata.get('clientid2')
# CLIENT_SECRET = userdata.get('clientsecret2')
# ACCESS_TOKEN = userdata.get('token2')

# oauth2 = OAuth2(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN)
# client = Client(oauth2)

# Introduction
This is an introduction to your report, you should edit this text/mardown section to compose. In this text/markdown, you should introduce:

*   Background of the problem
  * what type of problem: disease/readmission/mortality prediction,  feature engineeing, data processing, etc
  * what is the importance/meaning of solving the problem
  * what is the difficulty of the problem
  * the state of the art methods and effectiveness.
*   Paper explanation
  * what did the paper propose
  * what is the innovations of the method
  * how well the proposed method work (in its own metrics)
  * what is the contribution to the reasearch regime (referring the Background above, how important the paper is to the problem).

---

Obstructive sleep apnea hypopnea syndrome (OSAHS) is a breathing disorder where breathing is obstructed while sleeping (Loughlin Et al, 1996). Sleep apnea affects 1%-5% of children in the United States and can lead to other health illnesses if left untreated (Loughlin Et al, 1996; Marcus et al, 2012). Currently, at-home diagnostic tools for sleep apnea are only available for adults, leaving room for models to be created to address the needs for children (Fayyaz et al, 2023). The current state-of-the-art sleep apnea detection models created for adults (CNN (Chang et al., 2020), SE-MSCNN (Chen et al., 2022), CNN+LSTM (Zarei et al., 2022), Hybrid Transformer (Hu et al., 2022)) are effective but cannot be used for children because the sleep data differs between the two, and OSAHS symptoms for children require more attention (Choi et al, 2010; Gipson et al, 2019).

Polysomnography is commonly used to diagnose OSAHS. This process is used to collect various signals while sleeping such as brain activity (EEG), eye movement (EOG), heart rhythm (ECG), blood oxygen saturation (SpO2), blood CO2 levels (ETCO2), and air flow. Although polysomnography is the best method to diagnose OSAHS, it is complex, costly, intrusive, and requires clinician involvement (Spielmanns et al, 2019). Because of these issues, it is not easy for children and their families to use polysomnography to detect OSAHS at home. To address these issues, Fayyaz et al. propose a transformer-based model to help detect OSAHS in children.  Additionally, they compared using all the available polysomnography modalities to only a subset of the available modalities in the model, which is important because a subset of modalities may be significantly easier to collect at home, so finding if a subset performs as well as all polysomnography modalities data increases the feasibility of at-home detection. In terms of metrics, the proposed transformer-based model outperforms the current state-of-the-art sleep apnea detection models, and an additional edge of factoring demographic data into the modalities improves the proposed model’s performance even further.



# Scope of Reproducibility:

List hypotheses from the paper you will test and the corresponding experiments you will run.
---

The present study is based on the primary hypothesis that it is possible to achieve adult-level performance in detecting OSAHS. Specifically, through a custom transformer-based neural network, and its input in the form of preprocessed ECG and SPO2 signals, we hypothesize that we can effectively study and classify apnea-hypopnea in children.

# Methodology

This methodology is the core of your project. It consists of run-able codes with necessary annotations to show the expeiment you executed for testing the hypotheses.

The methodology at least contains two subsections **data** and **model** in your experiment.

In [ ]:
# we will be re-using the authors code for data pre-processing and loading, we will also extract some code for the model
!git clone https://github.com/healthylaife/Pediatric-Apnea-Detection.git

Cloning into 'Pediatric-Apnea-Detection'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 75 (delta 33), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (75/75), 31.40 KiB | 2.62 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
# rename cloned github folder for easier access to its functions

import os
source = 'Pediatric-Apnea-Detection/'
dest = 'PediatricApneaDetection/'
os.rename(source, dest)

In [ ]:
# import packages needed for data preprocessing, loading, and model training/testing

import numpy as np
from google.colab import drive
import os

import tensorflow as tf
import tensorflow_addons as tfa

import keras
from keras import Model
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.activations import sigmoid, relu
from keras.layers import Dense, Input, Conv1D, SeparableConvolution1D, concatenate, Layer, MultiHeadAttention, Add, LayerNormalization, Dropout, GlobalAveragePooling1D
from keras.regularizers import L2
from keras.losses import BinaryCrossentropy

from sklearn.utils import shuffle

In [ ]:
#check if gpu available.
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10609431299909879245
xla_global_id: -1
]


##  Data
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

---

The data is collected from the National Sleep Research Resource, an NHLBI-supported repository responsible for sharing large amounts of sleep data from various cohorts, clinical trials, and other data sources to advance sleep and circadian science. The two datasets we used can be found at these links: https://sleepdata.org/datasets/chat and https://sleepdata.org/datasets/nchsdb.

We utilize two different datasets, one from a Childhood Adenotonsillectomy Trial (CHAT)
And the other from the NCH Sleep Data Bank (NCHSDB).
The CHAT dataset is roughly 969 GB in size and was collected from 1,243 subjects ages 5-9 over a period of 5 years (2007-2012). The NCHSDB dataset is roughly 2.07 TB in size and was collected from 3,673 subjects ages 0-58 over a period of 2 years (2017-2019).
Both datasets will be pre-processed to only include necessary attributes before being loaded into the model for training and testing.
The raw data has been uploaded to a Box account for storage since the University of Illinois provides us with unlimited Box storage. Because of the large size of the data, we are currently in the process of pre-processing subsets of data and uploading the processed datasets to a separate folder to be loaded into our model.


**Due to large file sizes, we will only work on the CHAT dataset (only ~70GB available in Colab)**

**For Draft: we will only use 10 files to ensure model training completes before deadline.**

Two file types are used for the CHAT data construction, a \*.edf file containing time-series for multiple signals and a \*-nsrr.xml that contains annotations of the dataset. These annotations described events that happened during the study such as obstructive apnea, central apnea, hypopnea, SpO2 desaturation, EtCO2 artifact, limb movements, etc. These events include event type/concept (description), start time (onset), and duration.

For this model, we only consider apnea (Obstructive and Central grouped together) and hypopnea events.

In [ ]:
#Commented to allow rest of the nodebook to run. (No Data processing/loading/training for Submission)

# def downloadFromBox(filepath, file_content):
#   with open(filepath, "wb") as binary_file:
#     binary_file.write(file_content)

# #First Download processed data from box. (TODO)
# chat_out = '/content/chatprocessed/'
# os.makedirs(os.path.dirname(chat_out), exist_ok=True)

# processed_data = os.listdir(chat_out)

# #Second, download raw + annot data from BOX that is not already downloaded, not in badFiles list and is not in in processed (So we don't have to re-process data)
# chat_data = '/chatdata/'
# os.makedirs(os.path.dirname(chat_data), exist_ok=True)
# curr_downloaded = os.listdir(chat_data)

# #empirically known bad files (such as missing signals)
# badFiles = ["chat-baseline-300013.edf"]

# fields = [
#     'type',
#     'id',
#     'name',
# ]
# folder_raw = client.folder(folder_id='257515840362').get_items(fields=fields)
# folder_annot = client.folder(folder_id='257513450272').get_items(fields=fields)
# filenames = curr_downloaded

# i = 0
# for item in folder_raw:
#   if i>=10:
#     break
#   if not any(processed.startswith(item.name.split('.')[0]) for processed in processed_data) and item.name not in curr_downloaded and item.name not in badFiles:
#     print(f'download "{item.name}"')
#     file_content = client.file(item.id).content()
#     downloadFromBox(chat_data + item.name, file_content)
#     filenames.append(item.name)
#     i = i + 1
# print(filenames)
# for item in folder_annot:
#   if (item.name.split('-nsrr')[0] + '.edf') in filenames and item.name not in curr_downloaded:
#     print(f'download "{item.name}"')
#     file_content = client.file(item.id).content()
#     downloadFromBox(chat_data + item.name, file_content)

download "chat-baseline-300001.edf"
download "chat-baseline-300002.edf"
download "chat-baseline-300004.edf"
download "chat-baseline-300007.edf"
download "chat-baseline-300008.edf"
download "chat-baseline-300014.edf"
download "chat-baseline-300015.edf"
download "chat-baseline-300019.edf"
download "chat-baseline-300021.edf"
download "chat-baseline-300024.edf"
['chat-baseline-300001.edf', 'chat-baseline-300002.edf', 'chat-baseline-300004.edf', 'chat-baseline-300007.edf', 'chat-baseline-300008.edf', 'chat-baseline-300014.edf', 'chat-baseline-300015.edf', 'chat-baseline-300019.edf', 'chat-baseline-300021.edf', 'chat-baseline-300024.edf']
download "chat-baseline-300001-nsrr.xml"
download "chat-baseline-300002-nsrr.xml"
download "chat-baseline-300004-nsrr.xml"
download "chat-baseline-300007-nsrr.xml"
download "chat-baseline-300008-nsrr.xml"
download "chat-baseline-300014-nsrr.xml"
download "chat-baseline-300015-nsrr.xml"
download "chat-baseline-300019-nsrr.xml"
download "chat-baseline-300021-

## Annotation file conversion + Pre-processing
Annotation File is required to be in tsv format in the paper's provided code.
Since the Sleep data site only had this in XML, we had to convert it and change column names as shown below.

The Pre-processing code does the following:


*   Loads study using raw \*.edf and annoations \*-nsrr.tsv file using mne library. Annoations file is read in as a dataframe. (pandas read_cdv)
*   Checks if required channels are avaiable in the study. If not, it is discarded.
*   Finds event ids of apnea and hypopnea events in annoations
*   Select specific channels from raw file.
*   Signals are divided into equal length epochs (authors chose **30 EPOCH_LENGTH**),
*   Epochs are resampled to a **frequency of 128**.
*   For each Epoch, the intersection between the apnea events and hypopnea events are found in seconds, and appended to a labels_apnea and labels_hypopnea array. Essentially, these labels contains seconds of apnea and hypopnea, respectively, for each epoch.
*   The numpy array containing data, labels_apnea, and labels_hypopnea are saved.




In [ ]:
import pandas as pd
import io
import xml.etree.ElementTree as ET
import csv

def convert_xml_to_tsv(xml_file):

  # parse the xml file
  tree = ET.parse(xml_file)
  root = tree.getroot()

  # grab the relevant fields from the xml - Start (onset) , Duration (Duration), EventType (Description)
  fields = ['onset', 'duration', 'description']

  # create new tsv file
  tsv_file_name = xml_file.replace(".xml", ".tsv")

  # create csv writer object
  csv_writer = csv.writer(open(tsv_file_name, 'w'), delimiter='\t')

  # write the header row
  csv_writer.writerow(fields)

  # iterate over the xml elements and extract the data we want
  for element in root:

      if element.tag == "ScoredEvents":

        # this is all events
        for event in element:

          # this is single event
          for attr in event:

            if attr.tag == "Start":
              onset = attr.text

            if attr.tag == "Duration":
              duration = attr.text

            if attr.tag == "EventConcept":
              description = attr.text.split("|")[0]

          # for field in fields
          row = [onset, duration, description]

          # write row to csv file
          csv_writer.writerow(row)

In [ ]:
#Commented to allow rest of notebook to run. (No Data processing/loading/training for Submission)

# import glob
# import mne
# from PediatricApneaDetection.data.chat import preprocessing

# root = "/chatdata/"
# OUT_FOLDER = ''

# for edf_file in glob.glob(root + "*.edf"):
#         print("preprocessing " + edf_file)

#         annot_file = edf_file.replace(".edf", "-nsrr.xml")
#         convert_xml_to_tsv(annot_file)
#         annot_file_tsv = edf_file.replace(".edf", "-nsrr.tsv")

#         # preprocess data
#         shape = preprocessing.preprocess((edf_file, annot_file_tsv), preprocessing.identity, OUT_FOLDER)

#         print(f"final preprocessing shape: {shape}")

preprocessing /chatdata/chat-baseline-300002.edf
17:47:22 --- Processing chat-baseline-300002.edf


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    6.6s
[Parallel(n_jobs=8)]: Done 365 tasks      | elapsed:    7.6s
[Parallel(n_jobs=8)]: Done 5839 tasks      | elapsed:   15.1s
[Parallel(n_jobs=8)]: Done 13840 tasks      | elapsed:   20.4s
[Parallel(n_jobs=8)]: Done 24208 tasks      | elapsed:   28.7s
[Parallel(n_jobs=8)]: Done 27198 out of 27198 | elapsed:   30.6s finished


final preprocessing shape: 971
preprocessing /chatdata/chat-baseline-300007.edf


/content/PediatricApneaDetection/data/chat/preprocessing.py:98: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


17:48:50 --- Processing chat-baseline-300007.edf


...

final preprocessing shape: 955
preprocessing /chatdata/chat-baseline-300015.edf


/content/PediatricApneaDetection/data/chat/preprocessing.py:98: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


18:00:43 --- Processing chat-baseline-300015.edf


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 1016 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 3896 tasks      | elapsed:    5.7s
[Parallel(n_jobs=8)]: Done 7928 tasks      | elapsed:   12.8s
[Parallel(n_jobs=8)]: Done 13112 tasks      | elapsed:   19.9s
[Parallel(n_jobs=8)]: Done 19448 tasks      | elapsed:   30.2s
[Parallel(n_jobs=8)]: Done 20214 out of 20214 | elapsed:   31.3s finished


final preprocessing shape: 690


In [ ]:
# Commented to allow rest of notebook to run. (No Data processing/loading/training for Submission)
# moves npz files to processed folder

# root = '/content/'
# chat_out = '/content/chatprocessed/'
# os.makedirs(os.path.dirname(chat_out), exist_ok=True)

# for npz_file in glob.glob(root + "*.npz"):

#   print(npz_file)

#   dest = chat_out + npz_file.replace("/content/","").replace("\\","")

#   print(dest)

#   os.rename(npz_file, dest)

#upload to box (TODO)

/content/\chat-baseline-300002.edf_243_207.npz
/content/chatprocessed/chat-baseline-300002.edf_243_207.npz
/content/\chat-baseline-300014.edf_1069_763.npz
/content/chatprocessed/chat-baseline-300014.edf_1069_763.npz
/content/\chat-baseline-300021.edf_35_191.npz
/content/chatprocessed/chat-baseline-300021.edf_35_191.npz
/content/\chat-baseline-300007.edf_16_160.npz
/content/chatprocessed/chat-baseline-300007.edf_16_160.npz
/content/\chat-baseline-300004.edf_212_370.npz
/content/chatprocessed/chat-baseline-300004.edf_212_370.npz
/content/\chat-baseline-300015.edf_108_12.npz
/content/chatprocessed/chat-baseline-300015.edf_108_12.npz
/content/\chat-baseline-300019.edf_196_337.npz
/content/chatprocessed/chat-baseline-300019.edf_196_337.npz
/content/\chat-baseline-300001.edf_160_104.npz
/content/chatprocessed/chat-baseline-300001.edf_160_104.npz
/content/\chat-baseline-300024.edf_7_843.npz
/content/chatprocessed/chat-baseline-300024.edf_7_843.npz
/content/\chat-baseline-300008.edf_186_103.np

## CHAT Data Loading
Chat Data Loading is done as dollowed:

*   Path is provided containing processed \*.npz files (from previous steps)
*   Divide studies into folds (5 folds used here). For example, if 10 processed files are in the folder, each fold will have data from 2 processed files.
*   For each study in each fold, the signals, apnea labels, and hypopnea labels are loaded.
*   Then, the apnea labels and hypopnea labels are combined to y_c.
*   To reduce the size of the data and improve model training performance, negative sampling is conducted. This is done by getting indexes for where y_c == 0 (negative samples) and where y_c>0 (positive samples). Then, a ratio between number of positive_samples and negative_samples is used to determine how many negative samples should be kept. The index of the kept negative_samples is stored in negative_survived as shown below. Only the indexes in negative_survived and positive_samples are kept in the data.
*   Extract_rri is used to ensure ECG signal has equal length data points to other signals (EPOCH_LENGTH * FREQ). This is 30\*128=3840, which can be found in the model input size shown later.



In [ ]:
# due to hardcoded values, we had to copy the dataloader code and change it slightly to be able to run it
# from - https://github.com/healthylaife/Pediatric-Apnea-Detection/blob/main/data/chat/dataloader.py

import glob
import os
import random
import numpy as np
import pandas as pd
from scipy.signal import resample
from biosppy.signals.ecg import hamilton_segmenter, correct_rpeaks
from biosppy.signals import tools as st
from scipy.interpolate import splev, splrep

from PediatricApneaDetection.data.chat import dataloader

SIGS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
s_count = len(SIGS)

FREQ = 128
EPOCH_LENGTH = 30
ECG_SIG = 8

def load_data(path):
    # demo = pd.read_csv("../misc/result.csv")
    # ahi = pd.read_csv(r"C:\Data\AHI.csv")
    # ahi_dict = dict(zip(ahi.Study, ahi.AHI))
    root_dir = os.path.expanduser(path)
    file_list = os.listdir(root_dir)
    print(file_list)
    length = len(file_list)

    ################################### Fold the data based on number of respiratory events #########################
    study_event_counts = [i for i in range(0, length)]
    folds = []
    for i in range(5):
        folds.append(study_event_counts[i::5])

    x = []
    y_apnea = []
    y_hypopnea = []
    counter = 0
    for idx, fold in enumerate(folds):
        first = True
        for patient in fold:
            rri_succ_counter = 0
            rri_fail_counter = 0
            counter += 1
            print(counter)
            # for study in glob.glob(PATH + patient[0] + "_*"):
            study_data = np.load(path + file_list[patient - 1])
            signals = study_data['data']
            labels_apnea = study_data['labels_apnea']
            labels_hypopnea = study_data['labels_hypopnea']

            # identifier = study.split('\\')[-1].split('_')[0] + "_" + study.split('\\')[-1].split('_')[1]
            # demo_arr = demo[demo['id'] == identifier].drop(columns=['id']).to_numpy().squeeze()

            y_c = labels_apnea + labels_hypopnea
            neg_samples = np.where(y_c == 0)[0]
            pos_samples = list(np.where(y_c > 0)[0])
            ratio = len(pos_samples) / len(neg_samples)
            neg_survived = []
            for s in range(len(neg_samples)):
                if random.random() < ratio:
                    neg_survived.append(neg_samples[s])
            samples = neg_survived + pos_samples
            signals = signals[samples, :, :]
            labels_apnea = labels_apnea[samples]
            labels_hypopnea = labels_hypopnea[samples]

            data = np.zeros((signals.shape[0], EPOCH_LENGTH * FREQ, s_count + 2))
            for i in range(signals.shape[0]):  # for each epoch
                # data[i, :len(demo_arr), -3] = demo_arr
                data[i, :, -1], data[i, :, -2], status = dataloader.extract_rri(signals[i, ECG_SIG, :], FREQ,
                                                                     float(EPOCH_LENGTH))

                if status:
                    rri_succ_counter += 1
                else:
                    rri_fail_counter += 1

                for j in range(s_count):  # for each signal
                    data[i, :, j] = signals[i, SIGS[j], :]

            if first:
                aggregated_data = data
                aggregated_label_apnea = labels_apnea
                aggregated_label_hypopnea = labels_hypopnea
                first = False
            else:
                aggregated_data = np.concatenate((aggregated_data, data), axis=0)
                aggregated_label_apnea = np.concatenate((aggregated_label_apnea, labels_apnea), axis=0)
                aggregated_label_hypopnea = np.concatenate((aggregated_label_hypopnea, labels_hypopnea), axis=0)
            print(rri_succ_counter, rri_fail_counter)

        x.append(aggregated_data)
        y_apnea.append(aggregated_label_apnea)
        y_hypopnea.append(aggregated_label_hypopnea)

    return x, y_apnea, y_hypopnea


In [ ]:
# remove file to avoid issues in dataload
%rmdir /content/chatprocessed/.ipynb_checkpoints

rmdir: failed to remove '/content/chatprocessed/.ipynb_checkpoints': No such file or directory


In [ ]:
#Commented to allow rest of notebook to run (No Data processing/loading/training for Submission)

# PATH = chat_out
# OUT_PATH = '/content/chatloader/'
# os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)

# # load data
# x, y_apnea, y_hypopnea = load_data(PATH)
# # save data into .npz file
# for i in range(5):
#       print(x[i].shape, y_apnea[i].shape, y_hypopnea[i].shape)
#       np.savez_compressed(OUT_PATH + "chat_" + str(i), x=x[i], y_apnea=y_apnea[i], y_hypopnea=y_hypopnea[i])

# np.savez_compressed(OUT_PATH + "chat_1", x=x, y_apnea=y_apnea, y_hypopnea=y_hypopnea) #doesn't work because of mismatching shapes after first dimension



['chat-baseline-300015.edf_108_12.npz', 'chat-baseline-300004.edf_212_370.npz', 'chat-baseline-300008.edf_186_103.npz', 'chat-baseline-300019.edf_196_337.npz', 'chat-baseline-300014.edf_1069_763.npz', 'chat-baseline-300002.edf_243_207.npz', 'chat-baseline-300001.edf_160_104.npz', 'chat-baseline-300024.edf_7_843.npz', 'chat-baseline-300007.edf_16_160.npz', 'chat-baseline-300021.edf_35_191.npz']
[[0, 5], [1, 6], [2, 7], [3, 8], [4, 9]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
1
41 0
2
278 0
3
22 0
4
66 0
5
101 0
6
48 0
7
71 0
8
160 0
9
107 0
10
40 0
(319, 3840, 17) (319,) (319,)
(88, 3840, 17) (88,) (88,)
(149, 3840, 17) (149,) (149,)
(231, 3840, 17) (231,) (231,)
(147, 3840, 17) (147,) (147,)


In [ ]:
# !zip -r chatprocessed.zip chatprocessed/
# !zip -r chatloader.zip chatloader/

  adding: chatprocessed/ (stored 0%)
  adding: chatprocessed/chat-baseline-300015.edf_108_12.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300004.edf_212_370.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300008.edf_186_103.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300019.edf_196_337.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300014.edf_1069_763.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300002.edf_243_207.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300001.edf_160_104.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300024.edf_7_843.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300007.edf_16_160.npz (deflated 0%)
  adding: chatprocessed/chat-baseline-300021.edf_35_191.npz (deflated 0%)
  adding: chatloader/ (stored 0%)
  adding: chatloader/chat_2.npz (deflated 0%)
  adding: chatloader/chat_1.npz (deflated 0%)
  adding: chatloader/chat_4.npz (deflated 0%)
  adding: chatloader/chat_3.npz (deflated 0%)

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

---

Similar to what’s documented in the original paper, we will be implementing a model consisting of four components: segmentor, tokenizer, transformer, and multi-layer perceptron.

The segmentor will divide signals into equal-length epochs and forward them to the tokenizer. (This is done through pre-processing/dataloading steps shown previously.)

The tokenizer will construct tokenized representations of the segmentor’s output. Once these tokens have been generated, they will be passed to the transformer.
*   The tokenizer will handle regular and irregular time series data as well as data in tabular format. For consistency, data from all three formats will be resampled using a desired frequency (shown in pre-processing/dataloading).
*   Tokenizing can be seen in the model code below, between Input1 and before looping through transformer layers. Note that the Input shape is (Freq*Epoch_length, Num_signals). This input shapes needs to match what was created in preprocessing + dataloading steps.




The transformer will be constructed using five encoder modules. Each encoder module will consist of multi-head attention and a position-wise feed-forward network, supplemented by residual and normalization layers. The inspiration for each encoder module came from already established transformer architecture (Vaswani et al. 2017a); note we will not be using the decoder component from this architecture as it is typically used for generative tasks and thus is not needed for our model. The multi-head attention will consist of concatenated attention heads and a final fully connected layer to facilitate the model’s ability to focus on information across various representation sub-spaces. In terms of the position-wise feed- forward network, it will be comprised of one fully connected layer followed by a ReLU activation unit and then another fully connected layer. Output from the transformer unit will be forwarded to the multi-layer perceptron for analysis and prediction.

The multi-layer perceptron will be a two-layer fully connected network for forecasting the likelihood of an apnea-hypopnea event happening within a given epoch. The initial and subsequent layers of this network will consist of 256 and 128 neurons, respectively. Our model will use binary cross-entropy to determine loss.

**Draft Mistake** The model we created is actually the hybrid-transformer (Hu et al., 2022) that the author's compared their original model to. When checking the author's GitHub Repository, it looks like the last configuration they setup was for the comparison to the hybrid-transformer, which led to our confusion. Due to first draft deadline, we will not be able to train the author's model in time. For the final draft, we will train and test the author's original model (provided in line 109 https://github.com/healthylaife/Pediatric-Apnea-Detection/blob/main/models/models.py).

In [ ]:
# this function is provided in the paper's github repo but we included it here for clarity/describing key components
# from create_hybrid_transformer_model - https://github.com/healthylaife/Pediatric-Apnea-Detection/blob/main/models/transformer.py

class Patches(Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, input):
        input = input[:, tf.newaxis, :, :]
        batch_size = tf.shape(input)[0]
        patches = tf.image.extract_patches(
            images=input,
            sizes=[1, 1, self.patch_size, 1],
            strides=[1, 1, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches,
                             [batch_size, -1, patch_dims])
        return patches

class PatchEncoder(Layer):
    def __init__(self, num_patches, projection_dim, l2_weight):
        super(PatchEncoder, self).__init__()
        self.projection_dim = projection_dim
        self.l2_weight = l2_weight
        self.num_patches = num_patches
        self.projection = Dense(units=projection_dim, kernel_regularizer=L2(l2_weight),
                                bias_regularizer=L2(l2_weight))
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim)

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch)# + self.position_embedding(positions)
        return encoded

def mlp(x, hidden_units, dropout_rate, l2_weight):
    for _, units in enumerate(hidden_units):
        x = Dense(units, activation=None, kernel_regularizer=L2(l2_weight), bias_regularizer=L2(l2_weight))(x)
        x = tf.nn.gelu(x)
        x = Dropout(dropout_rate)(x)
    return x

#input shape used in paper (didn't work): ((60 * 32, 3))
#Used ((128*30,3)) which matches dataloader.
def create_model(input_shape):
    transformer_units = [32,32]
    transformer_layers = 2
    num_heads = 4
    l2_weight = 0.001
    drop_out= 0.25
    mlp_head_units = [256, 128]
    num_patches= 30
    projection_dim= 32

    input1 = Input(shape=input_shape)
    conv11 = Conv1D(16, 256)(input1)
    conv12 = Conv1D(16, 256)(input1)
    conv13 = Conv1D(16, 256)(input1)

    pwconv1 = SeparableConvolution1D(32, 1)(input1)
    pwconv2 = SeparableConvolution1D(32, 1)(pwconv1)

    conv21 = Conv1D(16, 256)(conv11)
    conv22 = Conv1D(16, 256)(conv12)
    conv23 = Conv1D(16, 256)(conv13)

    concat = concatenate([conv21, conv22, conv23], axis=-1)
    concat = Dense(64, activation=relu)(concat)
    concat = Dense(64, activation=sigmoid)(concat)
    concat = SeparableConvolution1D(32,1)(concat)
    concat = concatenate([concat, pwconv2], axis=1)

    ####################################################################################################################
    patch_size = input_shape[0] / num_patches

    normalized_inputs = tfa.layers.InstanceNormalization(axis=-1, epsilon=1e-6, center=False, scale=False,
                                                            beta_initializer="glorot_uniform",
                                                            gamma_initializer="glorot_uniform")(concat)

    patches = Patches(patch_size=patch_size)(normalized_inputs)
    encoded_patches = PatchEncoder(num_patches=num_patches, projection_dim=projection_dim, l2_weight=l2_weight)(patches)

    for i in range(transformer_layers):
        x1 = encoded_patches # LayerNormalization(epsilon=1e-6)(encoded_patches) # TODO
        attention_output = MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=drop_out, kernel_regularizer=L2(l2_weight),  # i *
            bias_regularizer=L2(l2_weight))(x1, x1)
        x2 = Add()([attention_output, encoded_patches])
        x3 = LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, transformer_units, drop_out, l2_weight)  # i *
        encoded_patches = Add()([x3, x2])

    x = LayerNormalization(epsilon=1e-6)(encoded_patches)
    x = GlobalAveragePooling1D()(x)
    #x = Concatenate()([x, demo])
    features = mlp(x, mlp_head_units, 0.0, l2_weight)

    logits = Dense(1, kernel_regularizer=L2(l2_weight), bias_regularizer=L2(l2_weight),
                   activation='sigmoid')(features)

    ####################################################################################################################

    model = Model(inputs=input1, outputs=logits)
    return model



In [ ]:
# model + training code is similar to https://github.com/healthylaife/Pediatric-Apnea-Detection/blob/main/train.py with slight adjustments and documentation

# Commented to let rest of model to run. (No Data processing/loading/training for Submission). We will be loading model from drive.
# model = create_model((128 * 30, 3))
loss_func = BinaryCrossentropy()
optimizer = "adam"

#Prevent over-fitting on same fold.
def lr_schedule(epoch, lr):

    if epoch > 50 and (epoch - 1) % 5 == 0:
        lr *= 0.5

    return lr

####Training
*   Load data from each fold and append to list.
*   For each fold, set y = 1 for any seconds of apnea/hypopnea events and adjust x to only contain required signals (currently only ["ECG", "SPO2"])
*   For each fold, we generate x_train and y_train based on data from all other folds. Then, the model is trainined on this set for 100 epochs. The epochs can be stopped early from early_stopper if loss isn't improving. LR_scheduled is used to reduce learning rate after 50 epochs to avoid over-fitting in the fold.
*   A model for each fold is created and saved.






In [ ]:
# training function for model

def train(config, fold):
  FOLD = fold
  x = []
  y = []
  for i in range(FOLD):
    data = np.load(config["data_path"] + str(i) + ".npz", allow_pickle=True)
    x.append(data['x'])
    y.append(data['y_apnea'] + data['y_hypopnea'])

  #x for specific channels
  # print(x.shape)
  x_chan = []
  #  np.zeros( (x.shape[0],x.shape[1],x.shape[2], len(config["channels"])))

  print(len(x))
  for i in range(FOLD):
    x[i], y[i] = shuffle(x[i], y[i])
    x[i] = np.nan_to_num(x[i], nan=-1)
    y[i] = np.where(y[i] >= 1, 1, 0)
    print(x[i].shape)
    #Select specific channels from data.
    x_chan.append(x[i][:, :, config["channels"]])
    print(x_chan[i].shape)

  print("training")
  for fold in range(FOLD):
    x_train, y_train = None, None
    for i in range(FOLD):
      if i != fold:
        if isinstance(x_train, np.ndarray):
          # x_train = x[i]
          # y_train = y[i]
          x_train = np.concatenate((x_train, x_chan[i]))
          y_train = np.concatenate((y_train, y[i]))
        else:
          # x_train = np.concatenate((x_train, x[i]))
          # y_train = np.concatenate((y_train, y[i]))
          x_train = x_chan[i]
          y_train = y[i]
    print(x_train.shape)
    print(y_train.shape)
    model.compile(optimizer=optimizer, loss=loss_func,metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

    # Early stopping stops training when
    # the training loss is no longer going down by much, so it's not worth it to continue training
    early_stopper = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    lr_scheduler = LearningRateScheduler(lr_schedule)
    model.fit(x=x_train, y=y_train, batch_size=512, epochs=config["epochs"], validation_split=0.1,
                    callbacks=[early_stopper, lr_scheduler])
    model.save(config["model_path"] + config["model_name"] + str(fold))
    keras.backend.clear_session()

  print("training complete")


In [ ]:
# from - https://github.com/healthylaife/Pediatric-Apnea-Detection/blob/main/main_chat.py with slight modifications
# Commented to allow for rest of notebook to run. (No Data processing/loading/training for Submission)

# data_path = '/content/chatloader/'
# model_path = '/content/model/'
# os.makedirs(os.path.dirname(model_path), exist_ok=True)

# sig_dict_chat = {
#     "EOG": [0, 1],
#     "EEG": [4, 5],
#     "ECG": [15,16],
#     "Resp": [9, 10],
#     "SPO2": [13],
#     "CO2": [14],
# }

# channel_list_chat = [
#     ["ECG", "SPO2"],
# ]

# for ch in channel_list_chat:
#     chs = []
#     chstr = ""
#     for name in ch:
#         chstr += name
#         chs = chs + sig_dict_chat[name]
#     print(chstr, chs)
#     config = {
#         "data_path": data_path + 'chat_',
#         "model_path": model_path,
#         "model_name": "model_hybrid_"+ chstr,
#         "regression": False,
#         "epochs": 100,  # best 200
#         "channels": chs,
#     }
#     train(config, 5)

ECGSPO2 [15, 16, 13]
5
(319, 3840, 17)
(319, 3840, 3)
(88, 3840, 17)
(88, 3840, 3)
(149, 3840, 17)
(149, 3840, 3)
(231, 3840, 17)
(231, 3840, 3)
(147, 3840, 17)
(147, 3840, 3)
training
(615, 3840, 3)
(615,)
Epoch 1/100
2/2 [==============================] - 52s 5s/step - loss: 1.2187 - precision: 0.5000 - recall: 0.0691 - val_loss: 1.1997 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 2/100
2/2 [==============================] - 3s 379ms/step - loss: 1.1982 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1804 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 3/100
2/2 [==============================] - 3s 421ms/step - loss: 1.1777 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1613 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 4/100
2/2 [==============================] - 3s 418ms/step - loss: 1.1558 - precision: 0.7143 - recall: 0.0182 - val_loss: 1.1423 - val_precision: 0.0000e+00 - val_rec

(846, 3840, 3)
(846,)
Epoch 1/100
2/2 [==============================] - 29s 15s/step - loss: 0.7629 - precision: 0.6901 - recall: 0.5870 - val_loss: 0.8144 - val_precision: 0.6042 - val_recall: 0.6591 - lr: 0.0010
Epoch 2/100
2/2 [==============================] - 4s 1s/step - loss: 0.7283 - precision: 0.6533 - recall: 0.7527 - val_loss: 0.8194 - val_precision: 0.5362 - val_recall: 0.8409 - lr: 0.0010
Epoch 3/100
2/2 [==============================] - 4s 1s/step - loss: 0.7322 - precision: 0.6164 - recall: 0.8995 - val_loss: 0.8031 - val_precision: 0.6038 - val_recall: 0.7273 - lr: 0.0010
Epoch 4/100
2/2 [==============================] - 4s 1s/step - loss: 0.7101 - precision: 0.6960 - recall: 0.7092 - val_loss: 0.7968 - val_precision: 0.6341 - val_recall: 0.5909 - lr: 0.0010
Epoch 5/100
2/2 [==============================] - 4s 1s/step - loss: 0.7180 - precision: 0.7548 - recall: 0.5353 - val_loss: 0.8070 - val_precision: 0.6364 - val_recall: 0.4773 - lr: 0.0010
Epoch 6/100
2/2 [====

(785, 3840, 3)
(785,)
Epoch 1/100
2/2 [==============================] - 26s 13s/step - loss: 0.7090 - precision: 0.6987 - recall: 0.6374 - val_loss: 0.7812 - val_precision: 0.6034 - val_recall: 0.8140 - lr: 0.0010
Epoch 2/100
2/2 [==============================] - 4s 1s/step - loss: 0.6946 - precision: 0.6461 - recall: 0.8275 - val_loss: 0.7846 - val_precision: 0.6182 - val_recall: 0.7907 - lr: 0.0010
Epoch 3/100
2/2 [==============================] - 4s 1s/step - loss: 0.6997 - precision: 0.6420 - recall: 0.7865 - val_loss: 0.7785 - val_precision: 0.6444 - val_recall: 0.6744 - lr: 0.0010
Epoch 4/100
2/2 [==============================] - 4s 1s/step - loss: 0.7198 - precision: 0.7090 - recall: 0.6199 - val_loss: 0.7824 - val_precision: 0.6111 - val_recall: 0.5116 - lr: 0.0010
Epoch 5/100
2/2 [==============================] - 4s 1s/step - loss: 0.7089 - precision: 0.7163 - recall: 0.5906 - val_loss: 0.7737 - val_precision: 0.6383 - val_recall: 0.6977 - lr: 0.0010
Epoch 6/100
2/2 [====

(703, 3840, 3)
(703,)
Epoch 1/100
2/2 [==============================] - 22s 9s/step - loss: 0.6501 - precision: 0.6752 - recall: 0.7524 - val_loss: 0.8217 - val_precision: 0.5091 - val_recall: 0.7568 - lr: 0.0010
Epoch 2/100
2/2 [==============================] - 3s 813ms/step - loss: 0.7271 - precision: 0.5845 - recall: 0.9111 - val_loss: 0.7651 - val_precision: 0.5667 - val_recall: 0.9189 - lr: 0.0010
Epoch 3/100
2/2 [==============================] - 3s 838ms/step - loss: 0.6952 - precision: 0.6390 - recall: 0.7587 - val_loss: 0.7305 - val_precision: 0.6486 - val_recall: 0.6486 - lr: 0.0010
Epoch 4/100
2/2 [==============================] - 3s 766ms/step - loss: 0.6773 - precision: 0.6950 - recall: 0.5714 - val_loss: 0.7348 - val_precision: 0.6190 - val_recall: 0.7027 - lr: 0.0010
Epoch 5/100
2/2 [==============================] - 3s 807ms/step - loss: 0.6502 - precision: 0.6881 - recall: 0.7143 - val_loss: 0.7366 - val_precision: 0.6383 - val_recall: 0.8108 - lr: 0.0010
Epoch 6/10

(787, 3840, 3)
(787,)
Epoch 1/100
2/2 [==============================] - 24s 11s/step - loss: 0.6720 - precision: 0.6611 - recall: 0.7746 - val_loss: 0.6958 - val_precision: 0.5897 - val_recall: 0.6970 - lr: 0.0010
Epoch 2/100
2/2 [==============================] - 4s 1s/step - loss: 0.6714 - precision: 0.6481 - recall: 0.7521 - val_loss: 0.6877 - val_precision: 0.6400 - val_recall: 0.4848 - lr: 0.0010
Epoch 3/100
2/2 [==============================] - 4s 1s/step - loss: 0.6629 - precision: 0.7574 - recall: 0.5803 - val_loss: 0.6941 - val_precision: 0.5714 - val_recall: 0.6061 - lr: 0.0010
Epoch 4/100
2/2 [==============================] - 4s 1s/step - loss: 0.6350 - precision: 0.7222 - recall: 0.6958 - val_loss: 0.7151 - val_precision: 0.5490 - val_recall: 0.8485 - lr: 0.0010
Epoch 5/100
2/2 [==============================] - 4s 1s/step - loss: 0.6341 - precision: 0.6752 - recall: 0.8141 - val_loss: 0.6970 - val_precision: 0.5714 - val_recall: 0.8485 - lr: 0.0010
Epoch 6/100
2/2 [====

training complete


In [ ]:
!zip -r model.zip model/

  adding: model/ (stored 0%)
  adding: model/model_hybrid_ECGSPO21/ (stored 0%)
  adding: model/model_hybrid_ECGSPO21/saved_model.pb (deflated 89%)
  adding: model/model_hybrid_ECGSPO21/keras_metadata.pb (deflated 94%)
  adding: model/model_hybrid_ECGSPO21/assets/ (stored 0%)
  adding: model/model_hybrid_ECGSPO21/variables/ (stored 0%)
  adding: model/model_hybrid_ECGSPO21/variables/variables.index (deflated 75%)
  adding: model/model_hybrid_ECGSPO21/variables/variables.data-00000-of-00001 (deflated 41%)
  adding: model/model_hybrid_ECGSPO21/fingerprint.pb (stored 0%)
  adding: model/model_hybrid_ECGSPO22/ (stored 0%)
  adding: model/model_hybrid_ECGSPO22/saved_model.pb (deflated 89%)
  adding: model/model_hybrid_ECGSPO22/keras_metadata.pb (deflated 94%)
  adding: model/model_hybrid_ECGSPO22/assets/ (stored 0%)
  adding: model/model_hybrid_ECGSPO22/variables/ (stored 0%)
  adding: model/model_hybrid_ECGSPO22/variables/variables.index (deflated 75%)
  adding: model/model_hybrid_ECGSPO22

# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


In [ ]:
#Download pre-trained model + loaded data for evaluation using gdown.
!gdown --fuzzy https://drive.google.com/file/d/1g_yxkCwi4L4hu2H1-mNGeFY9ymsr4nsz/view?usp=drive_link
!gdown --fuzzy https://drive.google.com/file/d/1CAtL7c4q1VpSUMeadIwyJIuL2_6aSiz9/view?usp=drive_link
!unzip model.zip
!unzip chatloader.zip

Downloading...
From: https://drive.google.com/uc?id=1g_yxkCwi4L4hu2H1-mNGeFY9ymsr4nsz
To: /content/model.zip
100% 18.8M/18.8M [00:00<00:00, 110MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1CAtL7c4q1VpSUMeadIwyJIuL2_6aSiz9
From (redirected): https://drive.google.com/uc?id=1CAtL7c4q1VpSUMeadIwyJIuL2_6aSiz9&confirm=t&uuid=12eea0e7-aaee-498b-84ad-1942808e0ddc
To: /content/chatloader.zip
100% 457M/457M [00:03<00:00, 119MB/s] 
Archive:  model.zip
   creating: model/
   creating: model/model_hybrid_ECGSPO21/
  inflating: model/model_hybrid_ECGSPO21/saved_model.pb  
  inflating: model/model_hybrid_ECGSPO21/keras_metadata.pb  
   creating: model/model_hybrid_ECGSPO21/assets/
   creating: model/model_hybrid_ECGSPO21/variables/
  inflating: model/model_hybrid_ECGSPO21/variables/variables.index  
  inflating: model/model_hybrid_ECGSPO21/variables/variables.data-00000-of-00001  
 extracting: model/model_hybrid_ECGSPO21/fingerprint.pb  
   creating: model/model_hybrid_ECGSPO

####Testing + Results Output
Each Model is tested using the that fold's data (since it wasn't used in the training of that model). x_test and y_test are setup similar to the training, except we only use the respective fold's data.

We use the author's metrics code to generate the results of the model. Each model's results are added to the results object. Once all fold's results are completed, the metrics are calculated and outputted.

In [ ]:
#Similar to https://github.com/healthylaife/Pediatric-Apnea-Detection/blob/main/test.py with some modications + comments
from PediatricApneaDetection.metrics import Result

def test(config, fold):
  FOLD = fold
  x = []
  y = []
  for i in range(FOLD):
    data = np.load(config["data_path"] + str(i) + ".npz", allow_pickle=True)
    x.append(data['x'])
    y.append(data['y_apnea'] + data['y_hypopnea'])

  #x for specific channels
  x_chan = []

  for i in range(FOLD):
    x[i], y[i] = shuffle(x[i], y[i])
    x[i] = np.nan_to_num(x[i], nan=-1)
    y[i] = np.where(y[i] >= 1, 1, 0)
    print(x[i].shape)
    #Select specific channels from data.
    x_chan.append(x[i][:, :, config["channels"]])
    print(x_chan[i].shape)

  print("test starting")
  result = Result()
  for i in range(FOLD):
    x_test = x_chan[i]
    y_test = y[i]
    model = tf.keras.models.load_model(config["model_path"] + config["model_name"] + str(i), compile=False)

    predict = model.predict(x_test)
    y_score = predict
    y_predict = np.where(predict > 0.5, 1, 0)

    result.add(y_test, y_predict, y_score)

  result.print()
  result.save(config["model_name"] + ".txt", config)

  del data, x_test, y_test, model, predict, y_score, y_predict

In [ ]:
#Test chat data

data_path = '/content/chatloader/'
model_path = '/content/model/'
os.makedirs(os.path.dirname(model_path), exist_ok=True)

sig_dict_chat = {
    "EOG": [0, 1],
    "EEG": [4, 5],
    "ECG": [15,16],
    "Resp": [9, 10],
    "SPO2": [13],
    "CO2": [14],
}

channel_list_chat = [
    ["ECG", "SPO2"],
]

for ch in channel_list_chat:
    chs = []
    chstr = ""
    for name in ch:
        chstr += name
        chs = chs + sig_dict_chat[name]
    print(chstr, chs)
    config = {
        "data_path": data_path + 'chat_',
        "model_path": model_path,
        "model_name": "model_hybrid_"+ chstr,
        "channels": chs,
    }
    test(config, 5)

ECGSPO2 [15, 16, 13]
(319, 3840, 17)
(319, 3840, 3)
(88, 3840, 17)
(88, 3840, 3)
(149, 3840, 17)
(149, 3840, 3)
(231, 3840, 17)
(231, 3840, 3)
(147, 3840, 17)
(147, 3840, 3)
test starting


10/10 [==============================] - 18s 2s/step


3/3 [==============================] - 4s 1s/step


5/5 [==============================] - 8s 2s/step


8/8 [==============================] - 11s 1s/step


5/5 [==============================] - 8s 1s/step
[75.23510971786834, 60.22727272727273, 63.08724832214765, 65.36796536796537, 52.38095238095239] 
[69.63350785340315, 81.81818181818183, 74.4186046511628, 59.11949685534591, 51.14503816793893] 
[86.36363636363636, 36.734693877551024, 42.10526315789473, 86.23853211009175, 91.78082191780823] 
[64.84848484848484, 89.74358974358975, 84.93150684931507, 46.72131147540984, 13.513513513513514] 
[77.10144927536233, 50.70422535211267, 53.78151260504202, 70.14925373134328, 65.68627450980392] 
[79.89374262101535, 63.73626373626373, 71.17880317231436, 75.15415852007821, 64.29100333209922] 
[74.20033910409988, 73.1232469559524, 74.49199948320818, 68.03032088965331, 68.73917589493807] 
Accuracy: 63.26 -+ 7.422 
Precision: 67.23 -+ 10.907 
Recall: 68.64 -+ 24.006 
Specifity: 59.95 -+ 27.810 
F1: 63.48 -+ 9.921 
AUROC: 70.85 -+ 6.230 
AUPRC: 71.72 -+ 2.768 
$ 63.3 \pm 7.4$& $67.2 \pm 10.9$& $68.6 \pm 24.0$& $63.5 \pm 9.9$& $70.9 \pm 6.2$& 


## Model Comparison

In [ ]:
# compare your model with others
# you don't need to re-run all other experiments, instead, you can directly refer the metrics/numbers in the paper

The ECG + SpO2 hybrid-transformer model had the following F1 and AUROC using the CHAT Dataset:
* F1: 63.48 -+ 9.921
* AUROC: 70.85 -+ 6.230

The original paper's ECG + SpO2  had the following F1 and AUROC using the CHAT Dataset (Found in Table 4 of the paper):
* F1: 78.8(0.4)
* AUROC: 84.9(0.7)

# Discussion/Analyses

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.

---

After looking through all the individual components of the original paper, we conclude that the paper is reproducible. Our current model performance might not be the same as what the original paper shows because we currently only processed a small subset of the data, however we are confident that once we utilize all the data available, we can achieve the same results as what is described in the original paper.

We will likely get closer to the author's results by re-implementing the model correctly instead of using the hybrid-transformer as we discussed in the models section in **Draft Mistake**.

The overall experience of reproducing the paper was satisfying as it had sections that were easy as well as difficult. The easiest section was understanding the overall flow of data throughout the high-level overview of the model. However, the actual implementation of the model was difficult because there were practical decisions that needed to be made. One such decision was where to store the data for easy use of preprocessing, training, and testing; we decided to upload all data to a Box account due to its ability to store large sizes of data and allow for easy connection and retrieval of data for machine learning tasks. Another difficult decision was determining resources that could be used to handle all the machine learning operations; Google Colab is currently being used, however if higher computational resources are needed we may utilize the University of Illinois at Urbana-Champaign’s campus research computing resources. One final difficult decision was whether we should utilize the same functions provided in the original paper’s GitHub; we decided that we could not use the same functions exactly as they were given but instead made some slight modifications to be able to work with our setup. In the process of reproducing the original paper, several helper functions needed to be implemented and many version-specific code libraries were utilized. In the future, these helper functions being available and explicit indications of the versions of libraries used will help improve reproducibility and allow individuals to focus more time on advancing the model and seeing better results.

#Final Draft Plans
In the next phase, we will first change the model we are using to the author's model to correct the mistake used in this first draft.

We will also use more CHAT data to train and test the model. We won't be able to use all the data due to Colab size limitations, however we should be able to use more than what we did in this draft.

We want to train models using all 6 signals as well as discussed in the paper (EOG, EEG, ECG, Resp, SpO2, CO2)

Create Figure comparing our 2 Signal (ECG, SpO2) model to the authors, as well as compare the 6 signal model.

As part of our ablations in our proposal, we will remove 1 transformer layer at a time to see how it affects the model.




# References

1. Fayyaz H, Strang A, Beheshti R. Bringing At-home Pediatric Sleep Apnea Testing Closer to Reality: A Multi-modal Transformer Approach. Proc Mach Learn Res. 2023 Aug;219:167-185. PMID: 38344396; PMCID: PMC10854997.

2. Choi Ji Ho, Kim Eun Joong, Choi June, Kwon Soon Young, Kim Tae Hoon, Lee Sang Hag, Lee Heung Man, Shin Choi, and Lee Seung Hoon. Obstructive sleep apnea syndrome: a child is not just a small adult. Annals of Otology, Rhinology & Laryngology, 119(10): 656–661, 2010.

3. Gipson Kevin, Lu Mengdi, and Kinane T Bernard. Sleep-disordered breathing in children. Pediatrics in review, 40(1):3, 2019.

4. Loughlin GM, Brouillette RT, Brooke LJ, Carroll JL, Chipps BE, England SJ, Ferber P, Ferraro NF, Gaultier C, Givan DC, et al. Standards and indications for cardiopulmonary sleep studies in children. American journal of respiratory and critical care medicine, 153 (2):866–878, 1996.

5. Marcus Carole L, Brooks Lee J, Ward Sally Davidson, Draper Kari A, Gozal David, Halbower Ann C, Jones Jacqueline, Lehmann Christopher, Schechter Michael S, Sheldon Stephen, et al. Diagnosis and management of childhood obstructive sleep apnea syndrome. Pediatrics, 130(3):e714–e755, 2012.

6. Spielmanns Marc, Bost David, Windisch Wolfram, Alter Peter, Greulich Tim, Nell Christoph, Storre Jan Henrik, Koczulla Andreas Rembert, and Boeselt Tobias. Measuring sleep quality and
efficiency with an activity monitoring device in comparison to polysomnography. Journal of clinical medicine research, 11(12):825, 2019.

7. Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Lukasz Kaiser, and Illia Polosukhin. Attention is all you need. Advances in neural information processing systems, 30, 2017b.

8. Hu Shuaicong, Cai Wenjie, Gao Tijie, and Wang Mingjie. A hybrid transformer model for obstructive sleep apnea detection based on self-attention mechanism using single-lead ecg. IEEE Transactions on Instrumentation and Measurement, 71:1–11, 2022.

